In [45]:
from arcgis.geometry import Envelope
from arcgis.gis import GIS

from pyproj import Proj, transform
import pandas as pd
import requests
import json

from IPython.display import Image, HTML, display

# Provide Credentials for the Copernicus Open Access Hub
Details on how to set up your free account can be found @ https://scihub.copernicus.eu/userguide/SelfRegistration

In [2]:
dhus_url = '***'
username = '***'
password = '***'

# Create Map for Extent Definition
* You will need to pan or zoom in the map before proceeding. The default extent is goofy and a "finished" notebook will handle the extent before and after the map has been manipulated.

In [96]:
ext_map = GIS().map('Boca de Acre, Brazil')
# ext_map.basemap = 'satellite'
ext_map

MapView(layout=Layout(height='400px', width='100%'))

# Convert Map Extent to WKT

In [87]:
map_bounds = Envelope(ext_map.extent).polygon

proj_a = Proj(init='epsg:3857')
proj_b = Proj(init='epsg:4326')

proj_coords = [transform(proj_a, proj_b, coord[0], coord[1]) for coord in map_bounds.coordinates()[0]]
proj_string = str(f"({tuple(['{} {}'.format(*coord) for coord in proj_coords ])})")
proj_string = proj_string.replace("'", "")
wkt_ext = f'POLYGON {proj_string}'
wkt_ext

'POLYGON ((-67.48144247103565 -8.571843813194329, -67.48144247103565 -8.503940214344922, -67.312527676114 -8.503940214344922, -67.312527676114 -8.571843813194329, -67.48144247103565 -8.571843813194329))'

# Format Query
More information on how to interact with the various API resources can be found at the following resources:
* https://scihub.copernicus.eu/userguide/OpenSearchAPI
* https://scihub.copernicus.eu/userguide/ODataAPI

In [88]:
geom = f'footprint:"Intersects({wkt_ext})"'
name = 'filename:S2A_*'
plat = 'platformname:Sentinel-2'
prod = 'producttype:S2MSI2A'  # Bottom-of-Atmosphere

clcv = 'cloudcoverpercentage:95'
igst = 'ingestiondate:[NOW-1DAY TO NOW]'

query = f"{' AND '.join([geom, name, plat, prod])}"
query

'footprint:"Intersects(POLYGON ((-67.48144247103565 -8.571843813194329, -67.48144247103565 -8.503940214344922, -67.312527676114 -8.503940214344922, -67.312527676114 -8.571843813194329, -67.48144247103565 -8.571843813194329)))" AND filename:S2A_* AND platformname:Sentinel-2 AND producttype:S2MSI2A'

# Submit Request to API

In [89]:
payload = {
    'orderby': 'ingestiondate desc',
    'format': 'json',
    'rows': 20,
    'q': query,
}

response = requests.get(dhus_url, params=payload, auth=(username, password))
print(f'Reponse Stats: {response.status_code}')

Reponse Stats: 200


# Ensure Products Were Returned Before Proceeding

In [90]:
results = response.json()['feed']

if 'entry' not in results.keys():
    print('No Results')
    
else:
    print(f'Found: {len(results["entry"])}')

Found: 20


# First Product Result
Observe how the various attributes of a given product are broken down into types; e.g. date, int, double, or str.

In [91]:
print(json.dumps(results['entry'][0], indent=2))

{
  "title": "S2A_MSIL2A_20190909T144731_N0213_R139_T19LFL_20190909T185341",
  "link": [
    {
      "href": "https://scihub.copernicus.eu/dhus/odata/v1/Products('f1727070-fb5e-4a75-b89d-35ee4106eb58')/$value"
    },
    {
      "rel": "alternative",
      "href": "https://scihub.copernicus.eu/dhus/odata/v1/Products('f1727070-fb5e-4a75-b89d-35ee4106eb58')/"
    },
    {
      "rel": "icon",
      "href": "https://scihub.copernicus.eu/dhus/odata/v1/Products('f1727070-fb5e-4a75-b89d-35ee4106eb58')/Products('Quicklook')/$value"
    }
  ],
  "id": "f1727070-fb5e-4a75-b89d-35ee4106eb58",
  "summary": "Date: 2019-09-09T14:47:31.024Z, Instrument: MSI, Mode: , Satellite: Sentinel-2, Size: 1.06 GB",
  "date": [
    {
      "name": "beginposition",
      "content": "2019-09-09T14:47:31.024Z"
    },
    {
      "name": "endposition",
      "content": "2019-09-09T14:47:31.024Z"
    },
    {
      "name": "ingestiondate",
      "content": "2019-09-09T23:42:45.771Z"
    }
  ],
  "int": [
    {
     

# Convert Results into a Data Frame

In [92]:
result_info = []

for result in results['entry']:
    
    guid = result['id']
    data = [l['href'] for l in result['link'] if len(l.keys()) == 1][0]
    size = [l['content'] for l in result['str'] if l['name'] == 'size'][0]
    look = [l['href'] for l in result['link'] if 'Quicklook' in l['href']][0]
    igst = [d['content'] for d in result['date'] if d['name'] == 'ingestiondate'][0]
    cdcv = [m['content'] for m in result['double'] if m['name'] == 'cloudcoverpercentage'][0]
    
    result_info.append({
        'guid': guid,
        'data': data,
        'size': size,
        'look': look,
        'igst': igst,
        'cdcv': cdcv
    })
    
result_df = pd.DataFrame(result_info)

# Set Time Fields
result_df['igst'] = pd.to_datetime(result_df['igst'])
result_df.head()

,guid,data,size,look,igst,cdcv
0,f1727070-fb5e-4a75-b89d-35ee4106eb58,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,1.06 GB,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,2019-09-09 23:42:45.771000+00:00,0.523306
1,09f71706-2646-4e06-bb5d-e7f2c5e67cde,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,1.07 GB,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,2019-08-30 23:39:23.896000+00:00,0.341976
2,bdf92034-5e44-4fff-944b-43fe0315e8e7,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,869.28 MB,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,2019-08-21 03:06:27.414000+00:00,85.203205
3,ca461179-e47f-433b-a29f-e7705e8c03d3,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,1.07 GB,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,2019-08-10 23:55:22.032000+00:00,2.557284
4,b9db1734-a0e8-41e5-bdff-5566c11dbfc6,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,1.06 GB,https://scihub.copernicus.eu/dhus/odata/v1/Pro...,2019-08-01 02:11:51.257000+00:00,0.008872


# Collect Most Recent Product w/ Least Cloud Cover

In [93]:
udates = list(result_df['igst'].map(lambda t: t.date()).unique())[0]

recent = result_df[result_df['igst'] >= udates.strftime('%Y-%m-%d')].copy()
recent.sort_values('cdcv', inplace=True)

target = recent.iloc[0]
target

guid                 f1727070-fb5e-4a75-b89d-35ee4106eb58
data    https://scihub.copernicus.eu/dhus/odata/v1/Pro...
size                                              1.06 GB
look    https://scihub.copernicus.eu/dhus/odata/v1/Pro...
igst                     2019-09-09 23:42:45.771000+00:00
cdcv                                             0.523306
Name: 0, dtype: object

# View Image & Download
* Download Target Image will let you pull down the actual product. 
* If Image does not disply, follow the URL to load your credentials into the browser, then refresh this cell. A "finished" notebook would pass your Open Access Hub Credentials and avoid this extra step.

In [94]:
display(Image(url=target.look))
display(HTML(f'''<a href="{target['data']}" target="_blank">Download Target Image</a>'''))